In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Retrieve the MOCs of P/SDSS9/color and II/337/vvv1

In [ ]:
from astroquery.cds import cds
from mocpy import MOC

In [ ]:
sdss9 = cds.find_datasets("ID=CDS/P/SDSS9/color", return_moc=True)
vvv1 = cds.find_datasets("ID=CDS/II/337/vvv1", return_moc=True)

`sdss9` and `II_337_vvv1` are `mocpy.MOC` objects. One can use them e.g. for computing their intersection. This will give us a new `mocpy.MOC` instance of maximum HEALPix order : $\max( order_{sdss9}, order_{II/337/vvv1})$

In [ ]:
moc = sdss9.intersection(vvv1)
moc.max_order

`mocpy.MOC` class has a `fill` method responsible for plotting the HEALPix cells of a MOC on a predefined matplotlib axe. This method accepts an `astropy.wcs.WCS` object along with a `matplotlib.axes.Axes` and some matplotlib style keyword arguments.

The next cell of code will define a matplotlib context and use it for drawing the `SDSS9` and `II/337/vvv1` MOCs along with their intersection MOC.

In [ ]:
import matplotlib.pyplot as plt
from mocpy import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
# MOCPy offers a way to easily create an WCS focusing in a specific skycoord with a
# FOV. The WCS class embeds internally a astropy.wcs.WCS object
# This define an ICRS aitoff projection.
fig = plt.figure(111, figsize=(10, 10))
with WCS(fig,
        fov=40 * u.deg,
        center=SkyCoord(270, -20, unit='deg', frame='icrs'),
        coordsys="icrs",
        rotation=Angle(0, u.degree),
        projection="AIT") as wcs:
    # Create an mpl axe
    ax = fig.add_subplot(1, 1, 1, projection=wcs)

    # Calls to `mocpy.MOC.fill` for each of the three MOCs with various mpl styling keywords
    # to differentiate them from each other.
    sdss9.fill(ax=ax, wcs=wcs, edgecolor='g', facecolor='g', linewidth=1.0, fill=True, alpha=0.5, label='P/SDSS9/color')
    vvv1.fill(ax=ax, wcs=wcs, edgecolor='r', facecolor='r', linewidth=1.0, fill=True, alpha=0.5, label='II/337/vvv1')
    moc.fill(ax=ax, wcs=wcs, edgecolor='b', facecolor='b', linewidth=1.0, fill=True, alpha=0.5, label='intersection')

plt.xlabel('ra')
plt.ylabel('dec')
plt.title('Intersection of P/SDSS9/color with II/337/vvv1')
plt.grid(color="black", linestyle="dotted")
plt.legend(loc='upper left')
plt.show()
plt.close()

At this point we have retrieved MOC objects from the MOCServer using `astroquery.cds`. Then we used `mocpy` to compute their intersection which gave us a new MOC.

We will use this MOC for filtering a vizier catalog. Querying the MOCServer using `astroquery.cds` can help us knowing which vizier tables have sources in our MOC. As `astroquery.cds.query_region` returns an `astropy.table.Table`, it is also possible to do some post-filtering on the large amount of tables we might receive.

------

## Retrieve a Vizier Table and filter it through a MOC

In [ ]:
data_cols = cds.query_region(moc)

As the returned object is an `astropy.table.Table`, one can use mask arrays to filter the table.

In [ ]:
mask = (data_cols['obs_regime'] == 'Optical') & \
    (data_cols['dataproduct_type'] == 'catalog') & \
    (data_cols['nb_rows'] < 10000) & (data_cols['nb_rows'] > 9000)
col = data_cols[mask]['obs_id', 'obs_title', 'dataproduct_type', 'cs_service_url', 'nb_rows']
col

Let's retrieve the `J/A+A/376/441/stars` table, filter it through our MOC and plot it to an aladin-lite view using ipyaladin.

First, let's use `astroquery.vizier` to get the full table.

In [ ]:
J_A_A_376_441_stars_id = (col['obs_id'] == 'J/A+A/376/441/stars')
catalog = col[J_A_A_376_441_stars_id]
from astroquery.vizier import Vizier

v = Vizier(columns=['*', '_RAJ2000', '_DEJ2000'])
v.ROW_LIMIT = -1
table = v.get_catalogs(catalog['obs_id'])[0]
table

We can filter this table to keep only the observations lying in the MOC. The sources from the filtered table are added to an aladin lite view along with the MOC.

In [ ]:
import ipyaladin as ipyal
aladin = ipyal.Aladin(target='17 56 -18 59', fov=15)
aladin

In [ ]:
# Add the MOC to the view
aladin.add_moc_from_dict(moc.serialize(format='json'), {
     'opacity': 0.5,
     'color': "#880000",
})

mask = moc.contains(table['_RAJ2000'].data * u.deg, table['_DEJ2000'].data * u.deg)
table_f = table[mask]
print(table_f)
aladin.add_table(table_f)

## A new TMOC class in MOCPy for Creating Temporal Multi-Order Coverage Maps

A new `TMOC` class allows the user to create, save and manipulate temporal MOCs. This feature has been merged in the master branch of MOCPy the 30th of May.

In the following we will retrieve the second table of the vizier catalog ``II/285`` and create an `astropy.time.Time` from its `JD` named column. The method `from_times` creates a TMOC from an `astropy.time.Time` object. We plot the TMOC and print some informations such as the maximum resolution of the TMOC, the first and last date of observation and the total duration of the observations.

In [ ]:
from astroquery.vizier import Vizier

viz = Vizier(columns=['*', '_RAJ2000', '_DEJ2000'])
viz.ROW_LIMIT = -1
table = viz.get_catalogs('II/285')[1]

print(table)

In [ ]:
from mocpy import TimeMOC
from astropy.time import Time

table_moc = TimeMOC.from_times(Time(table['JD'], format='jd', scale='tdb'))
table_moc.plot(title='2nd table of II/285')

# print characteristics such as the time of the first/last observations
print('Time of the first observation:', table_moc.min_time.iso)
print('Time of the last observation:', table_moc.max_time.iso)
# the total duration of the observation times
print('Total duration: {0} jd'.format(table_moc.total_duration.jd))
# the order of the TimeMoc
print('max order:', table_moc.max_order)